In [0]:
# データの用意
import numpy as np
import pandas as pd
import seaborn as sns

df = sns.load_dataset("iris")  # Irisのデータを取得
df.iloc[0, 1] = np.NaN  # わざと欠損値を作る

# データの表示
df.head()

In [0]:
# pandas-profilingのパッケージをインストール
!pip install pandas-profiling

# データのprofileを作成
import pandas_profiling as pdp
from IPython.display import HTML

profile = pdp.ProfileReport(df)
profile.to_file(outputfile="profile.html")
HTML(filename='profile.html')

In [0]:
# ノートブック上にグラフを描画するように設定
%matplotlib inline

# リスト5.2で欠損値にした箇所に対する計算エラーの警告を出力させないように設定
import warnings
warnings.simplefilter('ignore')

sns.boxplot(x='species', y='sepal_width', hue='species', data=df)
sns.pairplot(df, hue='species')

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold

# 将来的にデフォルト値が変更される警告が出力されないように設定
import warnings
warnings.simplefilter('ignore')

# 1. 前処理------
df = df.dropna()  # 欠損値のあるレコードを削除
del df['petal_width']  # データチェックの結果からpetal_lengthと相関が強いpetal_widthは削除

# カテゴリーデータであるspecies（種類）を数値データに変換
df['species'] = df['species'].map(
    {'setosa': 0, 'versicolor': 1, 'virginica': 2})

# Numpyの変数を用意
y = np.array(df['species'])
X = np.array(df.iloc[:, 0:3])  # 0, 1, 2行目の3つを取得

# 2. 入れ子式の交差検証法を行う用意
# 2.1 outer loopの設定
outer_loop = KFold(n_splits=5, shuffle=True, random_state=0)

# 2.2 inner loopの設定
inner_loop = KFold(n_splits=4, shuffle=True, random_state=0)

# 2.3 パイプライン生成
pipe = Pipeline([('scaler', StandardScaler()),
                 ('logistic', LogisticRegression())])

# 2.4 グリッドサーチの設定
# 比較するハイパーパラメータ設定
param_grid = {
    'logistic__C': [1, 10, 100],
}

gs = GridSearchCV(
    estimator=pipe, param_grid=param_grid, scoring='accuracy', cv=inner_loop)

# 3. 学習
val_result = np.zeros((5, 3))  # outer_loop数×ハイパーパラメータの種類
ol_index = 0  # outer_loopのindex

# outer loop
for train_val_index, test_index in outer_loop.split(X):
    X_train_val, X_test = X[train_val_index], X[test_index]
    y_train_val, y_test = y[train_val_index], y[test_index]

    # inner loop
    gs.fit(X_train_val, y_train_val)  # trainで学習し、valで評価
    val_result[ol_index] = gs.cv_results_["mean_test_score"]

    print(" outer loopの結果その{}：{}".format(ol_index+1,
                                         val_result[ol_index]))
    ol_index += 1

# 4. 評価
print("--\n outer loopの平均結果 ：{}\n--".format(val_result.mean(axis=0)))
# 出力すると3番目のハイパーパラメータ C=100が良いと分かる

# 5. テストデータで性能確認
clf_pipe = Pipeline([('scaler', StandardScaler()),
                     ('logistic', LogisticRegression(C=100))])
il_index = 0  # inner-loopのindex
test_result = np.zeros(5*4)  # outer_loop数×inner_loop数

# outer loop
for train_val_index, test_index in outer_loop.split(X):
    X_train_val, X_test = X[train_val_index], X[test_index]
    y_train_val, y_test = y[train_val_index], y[test_index]

    # inner loop
    for train_index, val_index in inner_loop.split(X_train_val):
        X_train, X_val = X_train_val[train_index], X_train_val[val_index]
        y_train, y_val = y_train_val[train_index], y_train_val[val_index]

        clf_pipe.fit(X_train, y_train)  # trainで学習し
        test_result[il_index] = clf_pipe.score(X_test, y_test)  # testの正解率を求める

        print(" inner loopの結果その{}：{}".format(il_index +
                                             1, test_result[il_index]))
        il_index += 1

print("--\n テストデータの平均正解率：{}".format(test_result.mean()))